# PSU DS 200 
# Lab 7A Evaluate Decision Trees and Understand Risk of Overfitting
## Learning Objectives 

In this lab, you will learn to be able to do the following:
- Learn to evaluate a predictive model regarding "how good is the model"
    * In particular, you will learn how to interpret "confusion matrix".
    * You will also learn how to identify "strength" and "weakness" of a predictive model from confusion matrix.
- You will gain a hands-on understanding about the important risk of a model's "overfitting" to 
training data (using NFL pass prediction problem and data) by comparing a model's prediction
performance for training data with that of testing data.
- You will also be able to compare the overfitting risk of two predictive models by comparing their prediction performance with their model complexity.

In [4]:
import numpy as np
import pandas as pd
import graphviz

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

## Reading Data for Constructing a Predictive Model
### Format of the Data
Data for constructing a predictive model is often stored in a "Comma-Separated Values" format, also commonly referred to as "CSV" format.  A file that stores data in CSV format uses ".csv" as its file extension.  

### Read_CSV in Panda
The easiest way to read a CSV file for the purpose of constructing a predictive model is to use read_csv function in Panda. The function reads data from a CSV file, and generates a table-style internal representation (called DataFrame).
<ul>
    <li> The first parameter of the function is the name of the csv file to be read. </li>

    <li> The "sep=" parameter in read_csv specifieds the separator of different fields.  Because the CSV file used for this lab uses comma (i.e., ",") as separator, the read_csv function has "sep="," as the second pamater. </li>
    
</ul>

In [15]:
Ben_pass= pd.read_csv('Ben-NE-9-10-2015-pass-6-1.csv', sep=",")

### Display the Data 
The first row of the CSV file contains the names of attributes/features for each passing play of a specific NFL game (between Pittsburgh Steelers and New England Patriots held on September 10th, 2015).  Because pass prediction model for each quaterback can be different, so this dataset is prepared for the purpose of predicting the outcome of passing play by Steeler quaterback Ben Rothlisberger. The last column of the CSV file contains the outcome of each passing play.

In general, the data for building a predictive model involves two types: 
<ol>
    <li> The input data to the model (which we also call FEATURES), and </li>
    <li> The DESIRED prediction output data of the model (for each corresponding input) </li>
    </ol>
Together, these data are used to "train" the model, as we shall see later, so that the model's output,
given each input, is AS CLOSE AS POSSIBLE (we will elaborate this point later) to the DESIRED output.

For example, the data for training to predict pass completion includes input data for the prediction (e.g., distance of pass, number of down, yards to go, etc), and the DESIRED output of the prediction (i.e., whether a pass plays completes or not). 

The meaning of all the input attributes/features for this lab are listed below:
<ul>
    <li>down: The number of down for the play</li>
    <li>ydstogo: The remaining yards needed to gain for a first down</li>
    <li>Yards.Gained.PrevPlay: The number of yards gained in the previous play (which can be a run play or passing play}</li>
    <li>AirYards: The distance between the starting position of the play and the receiving position of the passing play. A negative number means the quarterback moves back in the play such that the net yardage gain, if the pass completes, is negative. </li>
    <li>PassLocation: The area of passing location: -1 means left field, 0 means center field, 1 means right field. </li>
</ul>

The value of the desired prediction output (i.e., PassOutcome in the last column) in the dataset indicates whether the outcome of each passing play, represented by each row, is completes (i.e., having the value "1") or incomplete (i.e., having the value "0").  

In [16]:
print(Ben_pass)

    down  ydstogo  Yards.Gained.PrevPlay  AirYards  PassLocation  PassOutcome
0      1       10                     18        -4             1            1
1      1       10                      0         9             1            1
2      3       22                      6         1             1            1
3      1       10                      0         7            -1            1
4      1       10                     13         6            -1            1
5      1       10                     12         7            -1            1
6      1       10                      0         5             1            0
7      2       10                      0        25             1            0
8      3        5                     -1         6            -1            1
9      1       15                      4        -1             1            1
10     3       18                     -6        17            -1            1
11     1       20                      5         5            -1

## Prepare Labelled Data for Model Training
In order to use the data to train a predictitive model, we need to extract the input data and corresponding output data from the table-like internal representation (i.e., DataFrame) generated by Panda read_csv function (and subsequentenly assigned to the variable Ben_pass).  

One easy way to do this is to use .values method (of DataFrame) to 
<ol>
    <li> Extract values from the columns for the input data and assign it to a variable (e.g., "X" in the code below), and </li>
    <li> Extract values from the column for the DESIRED output data and assigned to a different variable (e.g., "Y" in the code below).</li>
</ol>

### Prepare Input Data for Model Training

We want to use the following five attributes as inputs for predicting pass completion:
<ul>
    <li> down: The number of down for the play </li>
    <li> ydstogo: The remaining yards needed to gain for a first down </li>
<li> Yards.Gained.PrevPlay: The number of yards gained in the previous play (which can be a run play or passing play} </li>
<li> AirYards: The distance between the starting position of the play and the receiving position of the passing play. A negative number means the quarterback moves back in the play such that the net yardage gain, if the pass completes, is negative. </li>
<li>PassLocation: The area of passing location: -1 means left field, 0 means center field, 1 means right field.</li>
    </ul>

In DataFrame, the first column is referred to as column "0" (rather than column 1).  Therefore, the columns of the table-like representation stored in Ben_pass are:
<ul>
    <li> The column
        storing the value of "down" is column 0. </li>
    <li> The column storing the value of "ydstogo" is column 1. </li>
    <li> The column storing the value of "Yards.Gained.PrevPlay" is column 2. </li>
    <li> The column storing the value of "AirYards" is column 3. </li>
    <li> The column storing the value of "PassLocation" is column 4. </li>
    <li> The column storing the value of desired model output "PassOutcome" is column 5. </li>
    </ul>

#### As we saw in Lab 5, to extract column 0, 1, 2, 3, and 4 from Ben_pass (which stores the table-like representation), we use "0:5" to indicate a range of columns from column 0 to (but NOT including) column 5. 

We store the extracted input data in a variable called X.

#### Notice that we do not want to extract column 5 because it contains the desired model output, so we want to save it in a different variable to prepare for model training (which takes model input data and desired model output data separately).

In [17]:
X= Ben_pass.values[:,0:5]
print(X)

[[ 1 10 18 -4  1]
 [ 1 10  0  9  1]
 [ 3 22  6  1  1]
 [ 1 10  0  7 -1]
 [ 1 10 13  6 -1]
 [ 1 10 12  7 -1]
 [ 1 10  0  5  1]
 [ 2 10  0 25  1]
 [ 3  5 -1  6 -1]
 [ 1 15  4 -1  1]
 [ 3 18 -6 17 -1]
 [ 1 20  5  5 -1]
 [ 2 11  9  4 -1]
 [ 2 13 -3 -2 -1]
 [ 3  6  7  6  0]
 [ 2  7  0 11  1]
 [ 1 10 13 16  1]
 [ 1 10 19  6  1]
 [ 2  8  2  0 -1]
 [ 3  8  0 39 -1]
 [ 3  3  1 19  1]
 [ 1 10  0 11  0]
 [ 2 10  0 17 -1]
 [ 1 10 18  7  0]
 [ 1  9  2  4 -1]
 [ 2  6  3  6  0]
 [ 1 10  0  5  1]
 [ 2 15  9  4 -1]
 [ 3  6  9 26  1]
 [ 2  8  2  0 -1]
 [ 3  3  5 23  1]
 [ 4  3  0  4  0]
 [ 2  5  5 26 -1]
 [ 3  5  0  7  0]
 [ 1 10  0  4  1]
 [ 3  5  1  5  1]
 [ 2  1  9 -1 -1]
 [ 2  1  0  0 -1]
 [ 1 10 28 23 -1]
 [ 1 10 13 -5 -1]
 [ 2 15 -5  7  0]
 [ 3  6 -1  2 -1]
 [ 1 10  0 25  1]
 [ 2  5  5  2  1]
 [ 2  8  0  1  1]
 [ 1  1  0  1  0]
 [ 1 10  0 18  1]
 [ 1 10  0 27  0]
 [ 3  5  5  4  0]
 [ 2  1  0  1  1]
 [ 3  1  0  1 -1]
 [ 2  3  7  9  0]
 [ 3  3  0  9  1]
 [ 2 12 -2 23  1]
 [ 3 12  0  6  0]
 [ 1 10  0

### Prepare Output Data for Model Training
To prepare desired output data (also referred to as target output data) for model training, we extract PassOutcome, which is stored in column 5 of Ben_pass table-like representation.  Similar to the previous step, we extract this data from ALL ROWS of the Ben_pass table-like representation. 

We store this output data in a variable Y.

In [18]:
Y= Ben_pass.values[:,5]
print(Y)

[1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 1 1 1 0 1 1 1]


As we can see from the result of print(Y), Y is a one-dimensional array.  

# Train a Decision Tree from a Set of Training Data

### Like Lab 5, we will use Python scikit-learn module to create a decision tree.
### Scikit-learn offers other machine learning models beyond decision trees.
### We do not need to install scikit-learn, because it has been pre-installed in Azure Notebooks (together with Python 3)
Scikit-learn (sklearn) is a Python module that contains many useful machine learning code, including code for constructing model from data, for creating predictive models using multiple machine learning methods (including decision tree which we will use), and for evaluating how good is the model created.  

### In the beginning of this notebook, we used the following import statement to import a submodule of scikit-learn (sklearn), tree (which provides functions/methods related to decision trees)
    from sklearn import tree

### Split the Data into Training Set and Testing Set using Train_Test_Split in sklearn
It is important that we reserve some data to evaluate how good is the prediction capability of the model.  This means that we are NOT going to use this reserved data to train the model.  THIS IS A VERY IMPORTANT POINT and we will return to it later for a more detail look.  For now, we will focus on the workflow for constructing a predictive model.

There are multiple ways to use data to train decision tree models.  We will use train_test_split Python command from sklearn Python module. In the beginning of this lab, we have imported train_test_split from sklearn as follows:

    from sklearn.model_selection import train_test_split

Importing a function from a library means we can directly use this function after the import.

Train_test_split has four parameters:
<ol>
    <li> The first paraemter is all of the model's input data. We have prepared variable X1 to be used here. </li>
    <li> The second parameter is all of the model's target output data.  We have prepared variable Y to be used here. </li>
    <li> The third parameter (test_size) is the percentage of the data (both model input and output) to be RESERVED for testing.  We use 25% here, and will revisit this topic later. </li>
    <li> The fourth paraemter (randome_state) is a random number to be used to randomly split the model's input and output data (i.e., the data stored in variable X and variable Y) into two subgroups: a Training Set and a Testing Set.
    </li></ol>

The output of the train_test_split function includes four inter-related results:
<ol>
    <li> The Model Input Data (features) used for Training: We store this in the variable X_train </li>
    <li> The Model Input Data (features) used for Testing: We store this in the variable X_test </li>
    <li> The Model Output Data (prediction) used for Training: We store this in the variable y_train </li>
    <li> The Model Output Data (prediction) used for Testing: We store this in the variable y_test </li>
    
    </ol>
    
### In this lab, we will split the data into training and testing through a 80% - 20% split.  As we will see later, this is one of the most common ways for splitting labelled data.

### Self-evaluation
You may notice the input to train_test_split is "X" here, rather than "X1" in Lab 5.  Do you know why?

In [19]:
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.20, random_state = 100)

In [20]:
print(X_test)

[[ 1 10 13 -5 -1]
 [ 1 10  0  5  1]
 [ 1 10 18  7  0]
 [ 1 10 18 39 -1]
 [ 1 10 28 23 -1]
 [ 3  8  2 13  0]
 [ 1 10  0 18  1]
 [ 1 20  5  5 -1]
 [ 2  8  2  0 -1]
 [ 3  4  3 -3  1]
 [ 2 27 -7 14  0]
 [ 1 10 12  7 -1]
 [ 3  3  1 19  1]
 [ 1 10 11 11 -1]]


### We also imported DecisionTreeClassifier from the tree submodule in sklearn:
    from sklearn.tree import DecisionTreeClassifier

### Using DecisionTreeClassifier to Train a Decision Tree
There are two steps involved in creating a decision tree using DecisionTreeClassifier.
1. Create an "empty" Decision Tree Classifier by specifying the various parameters we use to construct the decision tree. Save the created empty Decision Tree Classifier in a variable (e.g., "clf" in the code below) for the next step. Some of the important parameters are the following:
    * criterion: We will use 'entropy' as the criteria for creating decision trees in this class.  
    * max_depth: This parameter specifies how "deep" the decision tree is allowed.  If you choose "max_depth=3", the tree created will be limited to 3 levels. (The root node is at level 0.) As we will see later, the choice of this parameter can affect the model's risk for overfitting.
    * min_samples_leaf: This parameter specifies the minimum number of samples required in a leaf node. As we will see later, the choice of this parameter can also affect the model's risk for overfitting.
2. Use a set of input training data and output training data to "train" the Decision Tree Classifier (previously "empty" one; but will have an actual Decision Tree.

### Exercise 1 Train a Decision Tree (Model A) with a High Overfitting Risk (3 points)
In Lab 5, we chose the parameters of max_depth and min_samples_leaf.  In this lab,
we first intentionally create a decision tree with a high overfitting risk.

Fill in a large number for the parameters of max_depth (e.b., between 10 and 20) and 
set the min_samples_leaf to be 1. 

We will refer to this model you build as "Model A".

In [38]:
clf1 = DecisionTreeClassifier(criterion = 'entropy', random_state = 100,
                               max_depth=15, min_samples_leaf=1)
clf1.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=15,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

Execute the code below to visualize the decision tree. The only thing you need to fill in is the name of the 5th feature you used for input.  Replace '???' with a string of that feature name.

In [39]:
dot_data1= tree.export_graphviz(clf1, out_file=None, feature_names=('down','ydstogo','YardsGainedPrevPlay','AirYards','PassLocation'))

In [33]:
graph = graphviz.Source(dot_data1)
graph

You can save the visualization of decision tree in a PDF file using graph.render command below. Replace '????' with the name of the file you want to use for the PDF file (without the .pdf extension).  After executing the code below with the file name, you should be able to find the file (with .pdf extension) in the same Project folder as this notebook.

In [34]:
graph.render('dtree')

'dtree.pdf'

### Exercise 2 Evaluating Overfitting Risk of a Decision Tree Model (Model A) (5 points)
One way to understand the concept of "overfitting" for a decision tree (or other types of
machine learning model) is to see how the model's prediction performance of the training data 
compare with the prediction performance of testing data.
- (1) Uses the model (i.e., the decision tree learned) to predict pass completion for the training data. Generate the confusion matrix of the prediction results, and explain what it means (e.g., what are true positive, false positive, true negative, false negative). 
#### See the next text cell for instructions on interpreting confusion matrix
- (2) Use the model (i.e., the decision tree learned) to predict pass completion for the testing data. Generate the confusion matrix of the prediction results, and explain what it means. 
- (3) What is your hypothesis that may explain the significant difference between the prediction performance for the training data and the testing data?

### Answer to Exercise 2:
- (1) False Positive: 0, True Positive: 41 , True Negative: 15 , False Negative: 0
- (2) False Positive: 4, True Positive: 9 , True Negative: 0 , False Negative: 1
- (3) The reason that the training data is different than the testing data because when new data is introduced to the already trained prediction model, the output values will change. You can't use training data to evaluate.

In [40]:
prediction_training1=clf1.predict(X_train)

In [41]:
print(metrics.classification_report(y_train, prediction_training1))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        15
          1       1.00      1.00      1.00        41

avg / total       1.00      1.00      1.00        56



In [42]:
metrics.confusion_matrix(y_train, prediction_training1)

array([[15,  0],
       [ 0, 41]])

# Interpret Confusion Matrix

One of the effective and convenient ways to evaluate the prediction performance of a predictive model (whether the model is a decision tree or of other types) is to use confusion matrix.  Sklearn module
in Python includes a submodule called "metrics", one of the functions provided by the submodule is
confusion_matrix, which takes two parameters:
- the first parameter is the correct/desired output
- the second parameter is the prediction output generated by a model.

The output of confusion_matrix is a 2x2 array (often referred to as "confusion matrix"), such as 
the one you just generated above.
- The first row of the matrix corresponds to DESIRED/ACTUAL negative class (e.g., actual incomplete passes)
- The second row of the matrix corresponds to DESIRED/ACTUAL positive class (e.g., actual complete passes)
- The first column of the matrix corresponds to PREDICTED negative class (e.g., predicted incomplete passes)
- The second column of the matrix corresponds to PREDICTED positive class (e.g., predicted complete passes).

- Each entry in the array represents the total number of data with the corresponding predicted class and actual class. We also give each of them a more meaningful name:
- row 1, column 1: True Negative (correctly predicted negative class)
- row 1, column 2: False Positive (incorrect positive predictions, actually they are negative class)
- row 2, column 1: False Negative (incorrect negative predictions, actually they are positive class) 
- row 2, column 2: True Positive (correctly predicted positive class)

In [43]:
prediction_testing1 = clf1.predict(X_test)

In [44]:
print(prediction_testing1)

[1 0 1 1 1 1 1 1 1 1 1 1 1 1]


In [45]:
print(y_test)

[1 1 1 0 1 1 1 1 0 0 1 1 0 1]


In [46]:
print(metrics.classification_report(y_test, prediction_testing1))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         4
          1       0.69      0.90      0.78        10

avg / total       0.49      0.64      0.56        14



In [47]:
metrics.confusion_matrix(y_test, prediction_testing1)

array([[0, 4],
       [1, 9]])

# Compare two models regarding their overfitting risk

In order to have a fair comparison of two models regarding their overfitting risk, it is VERY important that we use the SAME set of training data and testing data.  Otherwise, one model may perform better than the other model because its testing data is "easier".  

We will construct a model with different parameters for a simpler decision tree, we will refer to this model as "Model B". We will then compare model B's performance and model complexity with model A's performance and model complexity regarding their overfitting risk.

### Exercise 3 Train a simpler Decision Tree (3 points)
Choose a max_depth between 5 and 6, set min_sample_leaf to 2.

In [48]:
clf2 = DecisionTreeClassifier(criterion = 'entropy', random_state = 100,
                               max_depth=5, min_samples_leaf=2)
clf2.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [49]:
dot_data2= tree.export_graphviz(clf2, out_file=None, feature_names=('down','ydstogo','YardsGainedPrevPlay','AirYards','PassLocation'))

In [50]:
graph = graphviz.Source(dot_data2)
graph

In [51]:
graph.render('Seconddecsiontree')

'Seconddecsiontree.pdf'

In [89]:
prediction_training2=clf2.predict(X_train)

In [74]:
print(prediction_training2)

[1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1
 0 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 0 1 1]


In [87]:
print(y_train)

[1 1 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1]


In [86]:
print(metrics.classification_report(y_train, prediction_training2))

             precision    recall  f1-score   support

          0       0.73      0.73      0.73        15
          1       0.90      0.90      0.90        41

avg / total       0.86      0.86      0.86        56



In [58]:
metrics.confusion_matrix(y_train, prediction_training2)

array([[11,  4],
       [ 4, 37]])

In [72]:
prediction_testing2 = clf2.predict(X_test)

In [60]:
print(prediction_testing2)

[1 1 1 0 0 0 1 1 1 1 1 1 0 1]


In [90]:
print(y_test)

[1 1 1 0 1 1 1 1 0 0 1 1 0 1]


In [92]:
print(metrics.classification_report(y_test, prediction_testing2))

             precision    recall  f1-score   support

          0       0.50      0.50      0.50         4
          1       0.80      0.80      0.80        10

avg / total       0.71      0.71      0.71        14



In [93]:
metrics.confusion_matrix(y_test, prediction_testing2)

array([[2, 2],
       [2, 8]])

### Exercise 4 Compare Two Models Regarding Overfitting Risk and Prediction Performance (9 points)

Based on the prediction performance of the second model (Model B), compare it with the prediction performance of the first model (Model A) by answering the following questions:
- (1) How does the prediction performance of Model B compared to Model A for TRAINING DATA?
- (2) How does the prediction performance of Model B compared to Model A for TESTING DATA?
- (3) Which model will you choose?
- (4) What do you think cause model A to predict worse than model B for testint data?
- (5) What lessons did you learn regarding ways to reduce the risk of overfitting?

### Answer to Exercise 4
- (1) The prediction performance for training data of 'model A' is better than 'model B' because 'Model A' has 56 true values, while 'Model B' only has 48
- (2)The prediction performance for testing data of 'model B' is better than 'model A' because 'Model B' has 9 true values, while 'Model A' only has 8
- (3) I will chose 'Model B' because the testing data is better to use than training data and Model 'B' has more true values
- (4) Model A predicts worse than model B because when new data is introduced to model a it handles the data worse than model b. Model A is fit to close to the data while model b can be generalized to new data better.
- (5) I learned that using testing data is better than using training data to compare values. I also learned that the model should not be fit to close to the data.